In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

C:\Users\elias\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Load the datasets
big_table_by_average = pd.read_csv('../frameaxis/big_table_by_average.tsv', sep='\t')
dev_labels = pd.read_csv('../data/data/en/dev-labels-subtask-2.txt', sep='\t', header=None, names=['article_id', 'frames'])

# Merge the datasets
merged_data = pd.merge(big_table_by_average, dev_labels, on='article_id')

merged_data.head()

,article_id,"('absorbent', 'nonabsorbent')","('atypical', 'typical')","('born', 'unborn')","('broad-minded', 'narrow-minded')","('inspiring', 'uninspiring')","('jawed', 'jawless')","('made', 'unmade')","('negative', 'positive')","('pointed', 'pointless')","('scrupulous', 'unscrupulous')",frames
0,813452859,0.080125,0.015796,-0.091310,0.022744,-0.070299,0.082165,-0.090066,-0.033324,0.056182,-0.122622,"Political,External_regulation_and_reputation,P..."
1,813494037,0.074769,0.024556,-0.083949,0.022723,-0.080870,0.076549,-0.082030,-0.036595,0.055352,-0.118175,"Political,Crime_and_punishment,External_regula..."
2,813547724,0.084561,0.007195,-0.087211,0.011782,-0.078827,0.083936,-0.094876,-0.032868,0.055550,-0.116927,"Political,Quality_of_life,External_regulation_..."
3,813552066,0.080229,0.019845,-0.095069,0.016930,-0.075646,0.080472,-0.088424,-0.026648,0.064943,-0.124149,"Public_opinion,Policy_prescription_and_evaluat..."
4,813601978,0.068588,0.026880,-0.090958,0.022179,-0.074495,0.081351,-0.082409,-0.025719,0.052067,-0.112090,"Political,Morality,Cultural_identity,Crime_and..."


In [4]:
# Transform the frames column to binary format
merged_data['frames'] = merged_data['frames'].str.split(',')
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(merged_data['frames'])

In [5]:
# Split the data into training and validation sets
X = merged_data.drop(columns=['article_id', 'frames'])
y = pd.DataFrame(binary_labels, columns=mlb.classes_)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Build the neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1408      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 14)                910       
                                                                 
Total params: 10,574
Trainable params: 10,574
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

Epoch 1/20
2/2 [==============================] - 1s 145ms/step - loss: 0.6956 - accuracy: 0.0000e+00 - val_loss: 0.6943 - val_accuracy: 0.0000e+00
Epoch 2/20
2/2 [==============================] - 0s 21ms/step - loss: 0.6933 - accuracy: 0.0294 - val_loss: 0.6923 - val_accuracy: 0.0000e+00
Epoch 3/20
2/2 [==============================] - 0s 22ms/step - loss: 0.6904 - accuracy: 0.0294 - val_loss: 0.6903 - val_accuracy: 0.0000e+00
Epoch 4/20
2/2 [==============================] - 0s 21ms/step - loss: 0.6883 - accuracy: 0.0000e+00 - val_loss: 0.6885 - val_accuracy: 0.0000e+00
Epoch 5/20
2/2 [==============================] - 0s 19ms/step - loss: 0.6858 - accuracy: 0.0294 - val_loss: 0.6868 - val_accuracy: 0.0000e+00
Epoch 6/20
2/2 [==============================] - 0s 20ms/step - loss: 0.6834 - accuracy: 0.0294 - val_loss: 0.6853 - val_accuracy: 0.0000e+00
Epoch 7/20
2/2 [==============================] - 0s 26ms/step - loss: 0.6792 - accuracy: 0.0588 - val_loss: 0.6837 - val_accuracy: 0